# Data Collection


#### Import libraries

In [24]:
!pip install torch_geometric

In [25]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import random
import os
from sklearn.preprocessing import StandardScaler
from torch_geometric.nn import GCNConv
import networkx as nx

# random.seed(42)
# torch.manual_seed(42)
# np.random.seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [26]:
# # Mount Drive
# from google.colab import drive
# drive.mount('/content/drive')

In [27]:
# # Install Kaggle CLI
# !pip install -q kaggle

In [28]:
# # Create folder & download the dataset
# import os

# dataset_dir = '/content/drive'
# # dataset_dir = '/content/drive/MyDrive/2025F/Cathay Hackathon'
# # os.makedirs(dataset_dir, exist_ok=True)

# csv_path = os.path.join(dataset_dir, 'flights.csv')

# if not os.path.exists(csv_path):
#     print("Downloading flights.csv from Kaggle (≈ 570 MB)…")
#     !kaggle datasets download -d usdot/flight-delays -f flights.csv -p {dataset_dir} --unzip
#     print("Download finished!")
# else:
#     print(f"{csv_path} already exists – skipping download.")

In [29]:


# Default: if you uploaded flights.csv directly
csv_path = "/content/flights.csv"

# If you uploaded a folder (e.g., "flight-delays/"), use:
# csv_path = "/content/flight-delays/flights.csv"

# Check if file exists
if not os.path.exists(csv_path):
    print(f'Error: File not found: {csv_path}, please upload the file "flights.csv"')
    print("\nAvailable file in the same repository directory")
else:
    print(f"Found: {csv_path}")
    print(f"File size: {os.path.getsize(csv_path) / 1e6:.1f} MB")

Found: /content/flights.csv
File size: 592.4 MB


# Download Data and Display

In [30]:
# Without Drive mounting
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download fabiendaniel/predicting-flight-delays-tutorial/input?select=flights.csv


mkdir: cannot create directory ‘/root/.kaggle’: File exists
cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 10, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/kaggle/cli.py", line 68, in main
    out = args.func(**command_args)
          ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/kaggle/api/kaggle_api_extended.py", line 1741, in dataset_download_cli
    with self.build_kaggle_client() as kaggle:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/kaggle/api/kaggle_api_extended.py", line 688, in build_kaggle_client
    username=self.config_values['username'],
             ~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^
KeyError: 'username'


In [31]:
# File verification

df = pd.read_csv(csv_path, nrows=5, low_memory=False)
print("First 5 rows:")
display(df.head())
print(f"Total rows: {len(pd.read_csv(csv_path)):,}")

First 5 rows:


,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,...,408,-22,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,...,741,-9,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,...,811,5,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,...,756,-9,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,...,259,-21,0,0,NaN,NaN,NaN,NaN,NaN,NaN


/tmp/ipython-input-656998792.py:6: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  print(f"Total rows: {len(pd.read_csv(csv_path)):,}")


Total rows: 5,819,079


# Data Preprocessing

In [32]:
# Data Preprocessing
print("Loading CSV...")
df = pd.read_csv(csv_path, nrows=5000000, low_memory=False)  # Use nrows for full data

# KEEP ONLY NON-CANCELLED/NON-DIVERTED
df = df[(df['CANCELLED'] == 0) & (df['DIVERTED'] == 0)].copy()

# DROP EXACT DUPLICATE FLIGHT RECORDS
duplicate_cols = ['YEAR', 'MONTH', 'DAY', 'AIRLINE', 'FLIGHT_NUMBER',
                  'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'SCHEDULED_DEPARTURE']
print(f"Before dedupe: {len(df)} rows")
df = df.drop_duplicates(subset=duplicate_cols)
print(f"After dedupe:  {len(df)} rows")

Loading CSV...
Before dedupe: 4908741 rows
After dedupe:  4908741 rows


# Build and Train GNN model with Evaluation










#### Build graph with real travel time as edge weight

In [33]:
# ------------------------------------------------------------------
# 1. BUILD GRAPH (clean + node features)
# ------------------------------------------------------------------
airports = list(set(df['ORIGIN_AIRPORT']) | set(df['DESTINATION_AIRPORT']))
airport_to_idx = {ap: i for i, ap in enumerate(airports)}
idx_to_airport = {i: ap for ap, i in airport_to_idx.items()}

# ---- Node features (same as before) --------------------------------
avg_dep    = df.groupby('ORIGIN_AIRPORT')['DEPARTURE_DELAY'].mean().reindex(airports).fillna(0)
avg_arr    = df.groupby('DESTINATION_AIRPORT')['ARRIVAL_DELAY'].mean().reindex(airports).fillna(0)
avg_wdelay = df.groupby('ORIGIN_AIRPORT')['WEATHER_DELAY'].mean().reindex(airports).fillna(0)
out_cnt    = df['ORIGIN_AIRPORT'].value_counts().reindex(airports).fillna(0)

scaler = StandardScaler()
node_features = scaler.fit_transform(
    np.column_stack([avg_dep, avg_arr, avg_wdelay, out_cnt])
).astype(np.float32)
features = torch.tensor(node_features, dtype=torch.float).to(device)   # [N, 4]

# ---- Edge index ----------------------------------------------------
src = df['ORIGIN_AIRPORT'].map(airport_to_idx).values
dst = df['DESTINATION_AIRPORT'].map(airport_to_idx).values
edge_index = torch.tensor(np.vstack((src, dst)), dtype=torch.long).to(device)   # [2, E]

# ---- Edge **travel time** (AIR_TIME + ARRIVAL_DELAY) ---------------
# If AIR_TIME is missing we impute the median per (origin,dest) pair
df['AIR_TIME'] = df['AIR_TIME'].fillna(
    df.groupby(['ORIGIN_AIRPORT','DESTINATION_AIRPORT'])['AIR_TIME'].transform('median')
)
df['AIR_TIME'] = df['AIR_TIME'].fillna(df['AIR_TIME'].median())

# Total minutes a passenger actually spends on this leg
df['TOTAL_MINUTES'] = df['AIR_TIME'] + df['ARRIVAL_DELAY'].clip(lower=0)

# Use the *average* total minutes per (origin,dest) as the true cost
edge_cost_series = df['TOTAL_MINUTES']  # one value per row (edge)
edge_cost = np.log1p(edge_cost_series.values)  # log-scale
edge_cost = (edge_cost - edge_cost.mean()) / (edge_cost.std() + 1e-8)
edge_cost = torch.tensor(edge_cost, dtype=torch.float).to(device)  # [E]


#### Edge-level GNN for total travel time prediction (log-scaled)

In [34]:
class RouteGNN(nn.Module):
    def __init__(self, node_dim=4, hidden=64):
        super().__init__()
        self.conv1 = GCNConv(node_dim, hidden)
        self.conv2 = GCNConv(hidden, hidden)
        self.scorer = nn.Sequential(
            nn.Linear(hidden * 2, hidden),
            nn.ReLU(),
            nn.Linear(hidden, 1)
        )

    def forward(self, x, edge_index):
        h = F.relu(self.conv1(x, edge_index))
        h = F.relu(self.conv2(h, edge_index))
        src, dst = edge_index
        edge_emb = torch.cat([h[src], h[dst]], dim=-1)
        return self.scorer(edge_emb).squeeze(-1)   # [E]

model = RouteGNN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=3e-3)
criterion = nn.MSELoss()

#### Training (80 epochs)

In [35]:
def train_one_epoch():
    model.train()
    optimizer.zero_grad()
    pred = model(features, edge_index)
    loss = criterion(pred, edge_cost)
    loss.backward()
    optimizer.step()
    return loss.item()

print("Training RouteGNN (predicting total travel time)…")
for epoch in range(1, 81):
    loss = train_one_epoch()
    if epoch <= 3 or epoch % 20 == 0:
        print(f"Epoch {epoch:2d} | MSE Loss: {loss:.5f}")

torch.save(model.state_dict(), 'route_gnn_trained_time.pth')
print("\nModel saved as 'route_gnn_trained_time.pth'")

Training RouteGNN (predicting total travel time)…
Epoch  1 | MSE Loss: 1.01401
Epoch  2 | MSE Loss: 1.00267
Epoch  3 | MSE Loss: 0.99480
Epoch 20 | MSE Loss: 0.85227
Epoch 40 | MSE Loss: 0.81567
Epoch 60 | MSE Loss: 0.79211
Epoch 80 | MSE Loss: 0.77587

Model saved as 'route_gnn_trained_time.pth'


### NetworkX graph with the learned edge costs

In [36]:
# ------------------------------------------------------------------
# 4. NetworkX graph (for Dijkstra) – POSITIVE WEIGHTS ONLY
# ------------------------------------------------------------------
G = nx.DiGraph()

model.eval()
with torch.no_grad():
    learned_scores = model(features, edge_index).cpu().numpy()   # log-scaled

# ---- recover the original scaling parameters --------------------------------
mu    = edge_cost.cpu().numpy().mean()
sigma = edge_cost.cpu().numpy().std()

# ---- build edges -----------------------------------------------------------
for i in range(edge_index.size(1)):
    u = edge_index[0, i].item()
    v = edge_index[1, i].item()
    z = learned_scores[i]                     # standardised log1p

    # ---- undo standardisation + log1p ------------------------------------
    minutes = np.expm1(z * sigma + mu)         # ≈ real minutes (can be <0 !)

    # ---- FORCE POSITIVE ----------------------------------------------------
    minutes = max(minutes, 1e-3)               # never zero/negative
    minutes += 1.0                            # tiny bias, keeps ordering

    G.add_edge(u, v, weight=minutes, minutes=minutes)

print(f"Graph built: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")
print(f"   min weight = {min(d['weight'] for _,_,d in G.edges(data=True)):.3f}")
print(f"   max weight = {max(d['weight'] for _,_,d in G.edges(data=True)):.1f}")

Graph built: 629 nodes, 8525 edges
   min weight = 1.001
   max weight = 3.4


## Rerouting simulation for finding the global shortest path

In [37]:
# --------------------------------------------------------------
# Helper: airport code → index  (you already have this)
# Helper: index → airport code  (you already have this)
# --------------------------------------------------------------
# Build a fast lookup: (origin, dest) → row with *fastest* flight
od_to_row = (
    df.loc[df.groupby(['ORIGIN_AIRPORT', 'DESTINATION_AIRPORT'])['SCHEDULED_TIME'].idxmin()]
      .set_index(['ORIGIN_AIRPORT', 'DESTINATION_AIRPORT'])
)

def fmt_time(t):
    """Convert any time representation to 'HHMM' (e.g. 624 → 0624)."""
    # t can be int, float, or str – force to str first
    s = str(int(float(t)))          # remove any decimal part, keep digits
    return s.zfill(4)               # pad with zeros on the left

# Testing

In [38]:
# Pick 100 random flights that actually exist in the data
flight_sample = df[['ORIGIN_AIRPORT','DESTINATION_AIRPORT','SCHEDULED_TIME', 'FLIGHT_NUMBER', 'AIRLINE', 'SCHEDULED_DEPARTURE', 'SCHEDULED_ARRIVAL']].drop_duplicates()
disrupted = flight_sample.sample(n=min(100, len(flight_sample)), random_state=70)

total_saved = success = 0.0
print("\n" + "="*90)
print(f"REROUTING EVALUATION – {len(disrupted)} CANCELLED FLIGHTS")
print("="*90)

for idx, row in disrupted.iterrows():
    o_ap = row['ORIGIN_AIRPORT']
    d_ap = row['DESTINATION_AIRPORT']
    orig_minutes = row['SCHEDULED_TIME']


    o_idx = airport_to_idx[o_ap]
    d_idx = airport_to_idx[d_ap]

    # ---- remember the original direct cost (if it exists) ----
    direct_data = G.get_edge_data(o_idx, d_idx)
    direct_cost = direct_data['minutes'] if direct_data else np.inf

    # ---- REMOVE the direct edge for the search -------------------------
    if direct_data:
        G.remove_edge(o_idx, d_idx)

    ## ---- find an alternative path (still the *cheapest* one) -----------------
    try:
        alt_path = nx.shortest_path(G, o_idx, d_idx, weight='weight')

        # ----- ORIGINAL cancelled flight (the row we are iterating) -----
        orig_flight   = str(row['AIRLINE'])+ str(row['FLIGHT_NUMBER'])         # e.g. "UA123"
        orig_dep      = fmt_time(row['SCHEDULED_DEPARTURE'])    # e.g. "08:00"
        orig_arr      = fmt_time(row['SCHEDULED_ARRIVAL'])      # e.g. "09:30"

        # ----- LAST hop of the alternative path ------------------------
        last_o = idx_to_airport[alt_path[-2]]
        last_d = idx_to_airport[alt_path[-1]]

        # pull the *fastest* flight on that O-D pair from the pre-computed dict
        alt_row = od_to_row.loc[(last_o, last_d)]

        alt_flight   = str(row['AIRLINE'])+ str(row['FLIGHT_NUMBER'])
        alt_dep      = fmt_time(alt_row['SCHEDULED_DEPARTURE'])
        alt_arr      = fmt_time(alt_row['SCHEDULED_ARRIVAL'])
        alt_minutes  = alt_row['SCHEDULED_TIME']

        # saved minutes
        saved = orig_minutes - alt_minutes
        alt_cost = saved

        # ----- build the path string ---------------------------------
        path_str = " → ".join(idx_to_airport[n] for n in alt_path)

    except nx.NetworkXNoPath:
        # no alternative → treat as failure
        saved = -np.inf
        alt_cost = -np.inf
        path_str = ""
        orig_flight = str(row['AIRLINE'])+ str(row['FLIGHT_NUMBER'])
        orig_dep     = row['SCHEDULED_DEPARTURE']
        orig_arr     = row['SCHEDULED_ARRIVAL']

    finally:
        # ---- restore the direct edge --------------------------------
        if direct_data:
            G.add_edge(o_idx, d_idx, **direct_data)

    # ---- print the required line ------------------------------------
    line_num = idx + 1
    if alt_cost >= 0:                     # success
        total_saved += saved
        success += 1
        print(f"{line_num:4d}. {o_ap}→{d_ap} | "
              f"Orig {orig_flight} {orig_dep} {orig_arr}\n"
              f"| Alt {alt_flight} {alt_dep} {alt_arr} via [{path_str}] | "
              f"Saved {saved:5.1f} min")
    else:
        print(f"{line_num:4d}. {o_ap}→{d_ap} | "
              f"Orig {orig_flight} {orig_dep} {orig_arr}\n"
              f"| No faster alternative")

print("="*90)
print(f"Success Rate        : {success/len(disrupted):.1%}")
print(f"Avg Saved per success: {total_saved/success:.1f} min" if success else "No savings")
print(f"Total Saved         : {total_saved:.1f} min")
print("="*90)


REROUTING EVALUATION – 100 CANCELLED FLIGHTS
2264743. SJC→LAS | Orig WN1011 1225 1345
| Alt WN1011 0755 0855 via [SJC → BUR → LAS] | Saved  20.0 min
1006770. TPA→PHX | Orig WN3184 1700 1955
| Alt WN3184 0635 0720 via [TPA → SDF → PHX] | Saved  70.0 min
4555179. 11618→12889 | Orig UA1537 1829 2117
| Alt UA1537 0530 0610 via [11618 → 13851 → 12889] | Saved 188.0 min
3073365. IAH→LAS | Orig UA195 0901 1014
| Alt UA195 1015 1125 via [IAH → RNO → LAS] | Saved 123.0 min
2890610. LAX→PHL | Orig AA598 0620 1440
| Alt AA598 0700 0846 via [LAX → IND → PHL] | Saved 214.0 min
35591. OAK→HOU | Orig WN1671 1025 1605
| Alt WN1671 2055 2345 via [OAK → ABQ → HOU] | Saved 110.0 min
1300291. MIA→EWR | Orig UA1703 1629 1928
| Alt UA1703 0540 0757 via [MIA → JAX → EWR] | Saved  42.0 min
4366775. EWR→MSP | Orig DL1338 1840 2050
| Alt DL1338 0850 1028 via [EWR → RIC → MSP] | Saved  32.0 min
4358921. ORD→DTW | Orig DL2017 1035 1252
| No faster alternative
250863. RDU→DTW | Orig DL1238 0915 1110
| Alt DL1238 